# Optimization Of Algo Case 2

Case 2: Instead of choosing stocks every day bases on some look back and holdin them for some n days; We now choose stocks at day I, hold for N days then close out, and buy new stocks at day I+N. The method of stock choosing is the same.

- Maximize Total Return from 2015-Now
- Minimize Sharpe Ratio: Avg Daily Return - Risk Free Rate(0.05) / STDEV(Daily Return)

- Paramters: 
    - Number of Stocks(NumStocks) int between (2,30)
    - How long to hold the stock(holdTime) int between (0,2)
    - Stop Loss float between (-0.01,-0.05)
    - How long to calculate the Percent Change from int between(2,7)

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pandas_market_calendars as mcal
import datetime
from Tools.Trade_Functions import *
import numpy as np
import mlflow
import optuna

/Users/atulkrishnan/Desktop/TradingAlgos/Stock_Picker_Algo


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
experiment_counter=0
mlflow.set_experiment("Momentum Rider Tuning:V4")
framedCal= get_calendar()

In [5]:
def objective(trial):
    global experiment_counter
    expirement_name = f"Expirement {experiment_counter}"
    
    with mlflow.start_run(run_name=expirement_name) as run:
        #Setup Paramter Variation
        
        TimeInput = trial.suggest_int('TimeInput', 2, 7)
        NumStocks= trial.suggest_int('NumStocks', 1, 15)
        holdTime=trial.suggest_int('holdTime', 1, 8)
        stopLoss=trial.suggest_float('stopLoss',-0.05, -0.01)
        numDaysToCheckStopLoss= trial.suggest_int('numDaysToCheckStopLoss', 1, holdTime)#must be less than holdtime
        TradeData=pd.DataFrame()
        
        
        
        #Date Range
        cal=list()
        DateRange = [None,None]
        for i in range(0,len(framedCal),holdTime+1):
            if DateRange[0]==None and framedCal[i].startswith(str(2015)):
                DateRange[0]=framedCal[i]
            if framedCal[i].startswith(str(2024)):
                DateRange[1]=framedCal[i]
            cal.append(framedCal[i])
        DateRange=[cal.index(DateRange[0]),cal.index(DateRange[1])]
        TotalReturnList=list()
        

        #Run Algorithim
        for day in cal[DateRange[0]:DateRange[1]]:
            dailyReturn,TradeData = pick_trade(TradeData,RunDate=day,NumStocks=NumStocks,stopLoss=stopLoss,TimeInput=TimeInput,holdTime=holdTime,debug=False,numDaysToCheckStopLoss=numDaysToCheckStopLoss)
            TotalReturnList.append(dailyReturn)
        TradeData=TradeData.reset_index(drop=True)
        returnDf = pd.DataFrame({'Date': pd.to_datetime(cal[DateRange[0]:DateRange[1]]),'Return':TotalReturnList})

        #Compound Retruns
        M = 1
        returnVal=[]
        for i in returnDf["Return"]:
            M = M*(1+i)
            returnVal.append(M)
        returnDf=pd.concat([returnDf,pd.DataFrame({"ReturnVals":returnVal})],axis=1)

        #Calculate Metrics
        SharpeRatio =(np.mean(returnDf["Return"]) * 252 - 0.05) / (np.std(returnDf["Return"]) * np.sqrt(252))
        CumulativeReturn = returnDf.loc[len(returnDf)-1,"ReturnVals"]
        max_drawdown = calculate_mdd(returnDf["Return"])
        sortino_ratio = calculate_sortino_ratio(returnDf["Return"])


        pLratio = len(returnDf["Return"][returnDf["Return"]>0])/ len(returnDf["Return"])

        avg_loss= np.mean(returnDf["Return"][returnDf["Return"]<0])
        avg_prof= np.mean(returnDf["Return"][returnDf["Return"]>0])
        


        #Saving A plot
        plt.figure(figsize=(15, 6))  # Set the figure size
        plt.plot(returnDf['Date'], returnDf['ReturnVals'],  linestyle='-', color='blue', label="Return Vals")
        plt.title("Return Values Over Time", fontsize=16)
        plt.xlabel("Date", fontsize=12)
        plt.ylabel("Return Vals", fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.legend(fontsize=12)
        plt.tight_layout()
        plt.savefig("Visulizations/returns.png", dpi=300, bbox_inches='tight')
        plt.close()
        #LOG Paramters, Metrics and Artifacts

        mlflow.log_param("TimeInput", trial.params['TimeInput'])
        mlflow.log_param("NumStocks", trial.params['NumStocks'])
        mlflow.log_param("holdTime", trial.params['holdTime'])
        mlflow.log_param("stopLoss", trial.params['stopLoss'])
        mlflow.log_param("numDaysToCheckStopLoss", trial.params['numDaysToCheckStopLoss'])

        
        mlflow.log_metric("CumulativeReturn", CumulativeReturn)
        mlflow.log_metric("SharpeRatio", SharpeRatio)
        mlflow.log_metric("Profit_Loss_Ratio", pLratio)
        mlflow.log_metric("avg_prof",avg_prof)
        mlflow.log_metric("avg_loss", avg_loss)
        
        mlflow.log_metric("Max_Draw_Down",max_drawdown)
        mlflow.log_metric("sortino_ratio",sortino_ratio)

        
        yearly_plots(returnDf)
        
        mlflow.log_artifact("Visulizations/returns.png", artifact_path="Visulizations")
        
        
        yearlyReturns=list()
        years=range(2015,2025)

        for year in range(2015,2025):
            returnDf_year = returnDf[returnDf['Date'].dt.year == year]
            returnDf_year.loc[:, 'Return'] = returnDf_year['Return'].astype(float)
            yearlyReturns.append(np.prod(1 + returnDf_year['Return']))
            mlflow.log_artifact(f"Visulizations/returns_{year}.png", artifact_path="Visulizations")
         
        plt.figure(figsize=(10, 6))
        plt.plot(years, yearlyReturns, marker='o', linestyle='-', color='blue', label='Annual Returns')

        # Add numeric values above each point
        for i, value in enumerate(yearlyReturns):
            plt.text(years[i], value, f'{value:.2f}', ha='center', va='bottom', fontsize=10)

        # Add labels, title, and grid
        plt.title("Annual Returns (2015-2024)", fontsize=16)
        plt.xlabel("Year", fontsize=12)
        plt.ylabel("Return Value", fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.legend(fontsize=12)
        plt.tight_layout()
        # Save the plot as PNG
        plt.savefig("Visulizations/annual_returns.png", dpi=300)
        
        mlflow.log_artifact("Visulizations/annual_returns.png", artifact_path="Visulizations")
        
        TradeData.to_csv("Visulizations/TradeData.csv")
        returnDf.to_csv("Visulizations/Returns.csv")
        mlflow.log_artifact("Visulizations/TradeData.csv", artifact_path="Generated_Data")
        mlflow.log_artifact("Visulizations/Returns.csv", artifact_path="Generated_Data")


    

            
        experiment_counter += 1
        
        return CumulativeReturn, SharpeRatio, sortino_ratio, max_drawdown

In [ ]:
study = optuna.create_study(directions=["maximize","maximize","maximize","minimize"])
study.optimize(objective, n_trials=1500)

[I 2024-11-28 22:26:52,100] A new study created in memory with name: no-name-f3c4a6aa-e2b1-4828-b920-35dabf68db70
[I 2024-11-28 22:26:59,550] Trial 0 finished with values: [0.10259368455137584, -1.5371806944742217, -0.1530622274093968, -0.902080585189727] and parameters: {'TimeInput': 7, 'NumStocks': 10, 'holdTime': 1, 'stopLoss': -0.04821618266637704, 'numDaysToCheckStopLoss': 1}.
[I 2024-11-28 22:27:05,428] Trial 1 finished with values: [0.48754492679095257, -1.0013883230292424, -0.10496872039261254, -0.6569836902889109] and parameters: {'TimeInput': 5, 'NumStocks': 15, 'holdTime': 4, 'stopLoss': -0.03157178631202725, 'numDaysToCheckStopLoss': 1}.
[I 2024-11-28 22:27:10,730] Trial 2 finished with values: [0.5967939039930422, -0.6902406733043173, -0.07322466733186542, -0.596286632947806] and parameters: {'TimeInput': 4, 'NumStocks': 10, 'holdTime': 6, 'stopLoss': -0.041710975306198876, 'numDaysToCheckStopLoss': 5}.
[I 2024-11-28 22:27:15,900] Trial 3 finished with values: [0.228585641

[I 2024-11-28 22:29:21,907] Trial 25 finished with values: [0.49095707987709014, -1.074648573686027, -0.11612926168015908, -0.600717432102448] and parameters: {'TimeInput': 4, 'NumStocks': 14, 'holdTime': 5, 'stopLoss': -0.03209524248773493, 'numDaysToCheckStopLoss': 3}.
[I 2024-11-28 22:29:27,528] Trial 26 finished with values: [0.08863663789454726, -2.0925431920095137, -0.23548311252186022, -0.9116331770824541] and parameters: {'TimeInput': 2, 'NumStocks': 5, 'holdTime': 3, 'stopLoss': -0.02957417103416608, 'numDaysToCheckStopLoss': 1}.
[I 2024-11-28 22:29:34,290] Trial 27 finished with values: [0.20631314348837582, -1.508227299965195, -0.14807263283862593, -0.826542922926587] and parameters: {'TimeInput': 7, 'NumStocks': 14, 'holdTime': 2, 'stopLoss': -0.04861998283325681, 'numDaysToCheckStopLoss': 2}.
[I 2024-11-28 22:29:41,812] Trial 28 finished with values: [1.374285167409462, 0.17393914427785795, 0.025378386589067048, -0.4789972419742787] and parameters: {'TimeInput': 5, 'NumSto

[I 2024-11-28 22:32:20,127] Trial 56 finished with values: [0.5985737248978772, -0.781370180978004, -0.09047966117808547, -0.5211022173839129] and parameters: {'TimeInput': 6, 'NumStocks': 14, 'holdTime': 4, 'stopLoss': -0.021567163754849374, 'numDaysToCheckStopLoss': 1}.
[I 2024-11-28 22:32:26,193] Trial 57 finished with values: [0.21204318236624323, -1.6961770253958888, -0.18722071954100114, -0.8105230174592203] and parameters: {'TimeInput': 7, 'NumStocks': 11, 'holdTime': 3, 'stopLoss': -0.03225247517958038, 'numDaysToCheckStopLoss': 1}.
[I 2024-11-28 22:32:32,039] Trial 58 finished with values: [0.4907027158979807, -0.8811777260610426, -0.09364533298116133, -0.5538283978646158] and parameters: {'TimeInput': 2, 'NumStocks': 13, 'holdTime': 4, 'stopLoss': -0.04458153530996661, 'numDaysToCheckStopLoss': 2}.
[I 2024-11-28 22:32:36,915] Trial 59 finished with values: [0.07418456019176838, -1.9102211250264411, -0.20474907339838797, -0.9579021679200544] and parameters: {'TimeInput': 4, 'N

[I 2024-11-28 22:35:19,036] Trial 87 finished with values: [0.5296140721537648, -1.1636970014909123, -0.13091484448762317, -0.5460358705825316] and parameters: {'TimeInput': 5, 'NumStocks': 12, 'holdTime': 8, 'stopLoss': -0.027090890126747598, 'numDaysToCheckStopLoss': 3}.
[I 2024-11-28 22:35:24,060] Trial 88 finished with values: [0.34825824639493747, -1.2896409350035611, -0.1389260588115649, -0.7614462489400706] and parameters: {'TimeInput': 4, 'NumStocks': 7, 'holdTime': 7, 'stopLoss': -0.04407647548713626, 'numDaysToCheckStopLoss': 7}.
[I 2024-11-28 22:35:28,848] Trial 89 finished with values: [0.040023839214359196, -2.1024035593035686, -0.2774340499188799, -0.9742378876782064] and parameters: {'TimeInput': 7, 'NumStocks': 1, 'holdTime': 8, 'stopLoss': -0.038680981111652926, 'numDaysToCheckStopLoss': 5}.
[I 2024-11-28 22:35:34,848] Trial 90 finished with values: [0.3268971368728397, -1.7408038249211337, -0.17428048039512326, -0.7218433113450248] and parameters: {'TimeInput': 7, 'Nu

[I 2024-11-28 22:38:10,557] Trial 117 finished with values: [0.822547357314982, 0.06561040755223117, 0.007441641889993291, -0.6568594396271998] and parameters: {'TimeInput': 2, 'NumStocks': 3, 'holdTime': 2, 'stopLoss': -0.0461526336903898, 'numDaysToCheckStopLoss': 1}.
[I 2024-11-28 22:38:17,173] Trial 118 finished with values: [0.20397901469305665, -1.557093149953308, -0.15033119728010746, -0.8038420289650867] and parameters: {'TimeInput': 6, 'NumStocks': 12, 'holdTime': 2, 'stopLoss': -0.04487078097773482, 'numDaysToCheckStopLoss': 2}.
[I 2024-11-28 22:38:22,661] Trial 119 finished with values: [0.46314502787448303, -0.7654906293199898, -0.09260279736303051, -0.700768033999471] and parameters: {'TimeInput': 4, 'NumStocks': 6, 'holdTime': 4, 'stopLoss': -0.025884898974989416, 'numDaysToCheckStopLoss': 3}.
[I 2024-11-28 22:38:27,755] Trial 120 finished with values: [0.34825824639493747, -1.2896409350035611, -0.1389260588115649, -0.7614462489400706] and parameters: {'TimeInput': 4, 'Nu

...

[I 2024-11-28 23:53:38,601] Trial 911 finished with values: [0.03036516798843019, -0.6709725223488939, -0.08454344954388694, -0.9823341577018767] and parameters: {'TimeInput': 7, 'NumStocks': 1, 'holdTime': 1, 'stopLoss': -0.03164858764227893, 'numDaysToCheckStopLoss': 1}.
[I 2024-11-28 23:53:43,472] Trial 912 finished with values: [0.006424201887584962, -2.3285355206052083, -0.2622913416836453, -0.9970017540904816] and parameters: {'TimeInput': 3, 'NumStocks': 1, 'holdTime': 8, 'stopLoss': -0.04407647548713626, 'numDaysToCheckStopLoss': 4}.
[I 2024-11-28 23:53:50,207] Trial 913 finished with values: [5.920471270292807, 0.7981139882464674, 0.18755332211685039, -0.43063155381348234] and parameters: {'TimeInput': 4, 'NumStocks': 1, 'holdTime': 1, 'stopLoss': -0.013476517263816538, 'numDaysToCheckStopLoss': 1}.
[I 2024-11-28 23:53:56,335] Trial 914 finished with values: [0.4406754871663816, -1.0189355401348688, -0.10723242526693838, -0.586781736861238] and parameters: {'TimeInput': 2, 'Nu

[I 2024-11-28 23:57:06,008] Trial 942 finished with values: [1.3509899378229715, 0.40129373626918263, 0.06133407766537774, -0.6420746004133431] and parameters: {'TimeInput': 3, 'NumStocks': 1, 'holdTime': 2, 'stopLoss': -0.01272775097917396, 'numDaysToCheckStopLoss': 2}.
[I 2024-11-28 23:57:11,591] Trial 943 finished with values: [0.15601818287778266, -1.6449723341731652, -0.1820269668066818, -0.864904418489007] and parameters: {'TimeInput': 6, 'NumStocks': 5, 'holdTime': 5, 'stopLoss': -0.048902369048140125, 'numDaysToCheckStopLoss': 4}.
[I 2024-11-28 23:57:16,650] Trial 944 finished with values: [0.8921591418664375, 0.6082505798510549, 0.09553815843065126, -0.9005285217461745] and parameters: {'TimeInput': 4, 'NumStocks': 1, 'holdTime': 7, 'stopLoss': -0.048902369048140125, 'numDaysToCheckStopLoss': 2}.
[I 2024-11-28 23:57:22,362] Trial 945 finished with values: [0.29800235852319024, -0.8097012086544182, -0.08995130215510942, -0.7837307871805907] and parameters: {'TimeInput': 3, 'Num

[I 2024-11-29 00:00:02,168] Trial 973 finished with values: [0.1299618323882872, -0.5040625177779305, -0.05364382113967509, -0.9691886167461582] and parameters: {'TimeInput': 2, 'NumStocks': 1, 'holdTime': 8, 'stopLoss': -0.046343028466212216, 'numDaysToCheckStopLoss': 6}.
[I 2024-11-29 00:00:07,354] Trial 974 finished with values: [0.44533788014102405, -0.0013582106816919122, -0.00023675791997208106, -0.781155198186268] and parameters: {'TimeInput': 2, 'NumStocks': 1, 'holdTime': 5, 'stopLoss': -0.02634303721960878, 'numDaysToCheckStopLoss': 2}.
[I 2024-11-29 00:00:13,750] Trial 975 finished with values: [0.13453572703656638, -0.5452238815176773, -0.10388092937657302, -0.9565484066785925] and parameters: {'TimeInput': 6, 'NumStocks': 1, 'holdTime': 1, 'stopLoss': -0.015041884475840907, 'numDaysToCheckStopLoss': 1}.
[I 2024-11-29 00:00:19,730] Trial 976 finished with values: [0.15069924344315966, -1.9753103097179687, -0.19570956450630014, -0.8546355220343335] and parameters: {'TimeInpu

[I 2024-11-29 00:02:50,325] Trial 1003 finished with values: [1.290643287344672, 0.3613464156009408, 0.04870305106565152, -0.7334211672848268] and parameters: {'TimeInput': 4, 'NumStocks': 1, 'holdTime': 1, 'stopLoss': -0.029169404461554128, 'numDaysToCheckStopLoss': 1}.
[I 2024-11-29 00:02:55,284] Trial 1004 finished with values: [1.234917791626335, 0.557347623019341, 0.1747292183288803, -0.5529659705577487] and parameters: {'TimeInput': 4, 'NumStocks': 1, 'holdTime': 8, 'stopLoss': -0.011079942403128769, 'numDaysToCheckStopLoss': 2}.
[I 2024-11-29 00:03:00,198] Trial 1005 finished with values: [1.4107764755786139, 0.6855193763323247, 0.16804281616031178, -0.7154916644909224] and parameters: {'TimeInput': 2, 'NumStocks': 1, 'holdTime': 7, 'stopLoss': -0.011079942403128769, 'numDaysToCheckStopLoss': 3}.
[I 2024-11-29 00:03:06,052] Trial 1006 finished with values: [0.10882536483729835, -0.270081218580736, -0.030645166655103476, -0.9535021817927392] and parameters: {'TimeInput': 3, 'NumS

[I 2024-11-29 00:05:40,593] Trial 1033 finished with values: [0.7620365288758586, 0.06981906758521458, 0.014598500988583545, -0.5975853213826036] and parameters: {'TimeInput': 6, 'NumStocks': 1, 'holdTime': 2, 'stopLoss': -0.011079942403128769, 'numDaysToCheckStopLoss': 1}.
[I 2024-11-29 00:05:46,129] Trial 1034 finished with values: [0.2395175405960699, -0.5211217708743181, -0.05709326800836087, -0.8316857377239284] and parameters: {'TimeInput': 4, 'NumStocks': 2, 'holdTime': 3, 'stopLoss': -0.048902369048140125, 'numDaysToCheckStopLoss': 3}.
[I 2024-11-29 00:05:51,077] Trial 1035 finished with values: [0.04967515729807929, -1.443461585088891, -0.14964302975445581, -0.9604963577135857] and parameters: {'TimeInput': 6, 'NumStocks': 1, 'holdTime': 8, 'stopLoss': -0.03454632752121514, 'numDaysToCheckStopLoss': 3}.
[I 2024-11-29 00:05:58,660] Trial 1036 finished with values: [1.5185167593913824, 0.2705337742254141, 0.037832893489254546, -0.3811127609478895] and parameters: {'TimeInput': 3